# Generate configuration using the `revisitpy` package

Source: https://revisit.dev/docs/revisitpy/

Also: 
- https://github.com/revisit-studies/revisitpy-examples
- https://github.com/revisit-studies/revisitpy

In [42]:
import revisitpy as rvt
import pandas as pd 
import polars as pl
from datetime import date 
import json
# import revisitpy_server as rvt_server

In [2]:
# confirm working directory 

import os 
os.getcwd()

'/Users/shenglong/Downloads/study'

A revisit study has the following components in its `config.json`: [original link](https://revisit.dev/docs/typedoc/interfaces/StudyConfig/#importedlibraries)  

- `$schema`: ...
- `studyMetadata`: ...
- `uiConfig`: ...
- `importedLibraries`: ...
- `components`: ...
- `sequence`: ...

---

Some other things: 

- [`dataclasses`](https://docs.python.org/3/library/dataclasses.html): something that is related to python
- [`dataclasses.asdict`](https://docs.python.org/3/library/dataclasses.html): converts the dataclass obj to a dict

They are creating a specific dataclass obj `DataRow`: [link](https://github.com/revisit-studies/revisitpy/blob/51414e51d4c1f9c1f66b3f9c642c3c40a60138fc/src/revisitpy/revisitpy.py#L559)

```python
# Create a data class with attributes based on the headers
        DataRow = make_dataclass("DataRow", [(header, Any) for header in headers])
```

## Metadata 

- Example here: https://revisit.dev/docs/revisitpy/examples/example_jnd_study/ 

In [21]:
# Meta Data
study_metadata = rvt.studyMetadata(
    authors=["Sheng Long"],
    organizations=["Northwestern University"],
    title='Retrieve Value Judgment Study',
    description='',
    date=date.today().strftime("%Y-%m-%d"),
    version='1.0'
)

# UI Config
ui_config = rvt.uiConfig(
  contactEmail="shenglong@u.northwestern.edu",
  logoPath="assets/revisitLogoSquare.svg",
  withSidebar=True,
  withProgressBar=False,
  nextOnEnter=True,
  minWidthSize=800,
  minHeightSize=800,
)

# print(study_metadata)
# print(ui_config)

In [4]:
# read in external data 

ret_df = pl.read_parquet('public/vis-decode-retrieve-value/encqa_v1_ret.parquet')
# add a column for the id 
ret_df = ret_df.with_row_index("id", offset=1)
ret_df.head()

print(ret_df.select(pl.col('chart_spec')).head(1).item())

{"config": {"view": {"continuousWidth": 300, "continuousHeight": 300}, "axis": {"grid": false}, "bar": {"color": "gray"}}, "data": {"name": "data-9a1c73a21da12f2e668e101efeeb1bbd"}, "mark": {"type": "point", "color": "gray", "filled": true, "size": 300}, "encoding": {"x": {"axis": {"labelAngle": 0, "title": null}, "field": "cat", "type": "nominal"}, "y": {"aggregate": "mean", "axis": {"title": "Var"}, "field": "var1", "type": "quantitative"}}, "height": 400, "width": 400, "$schema": "https://vega.github.io/schema/vega-lite/v5.17.0.json", "datasets": {"data-9a1c73a21da12f2e668e101efeeb1bbd": [{"cat": "C", "var1": 94.53337893746139, "var2": 45.7573094474521}, {"cat": "E", "var1": 65.92327500197413, "var2": 53.03417406992416}, {"cat": "D", "var1": 52.33420881465709, "var2": 54.67417687172758}, {"cat": "B", "var1": 56.60850125771311, "var2": 43.262914732152254}, {"cat": "A", "var1": 40.84536899383492, "var2": 56.02464150248745}, {"cat": "E", "var1": 49.95558389337083, "var2": 35.4335154793

## Generate component related 

In [30]:
# Introduction
introduction = rvt.component(type='markdown', 
                             path='vis-decode-retrieve-value/assets/introduction.md', 
                             component_name__= 'introduction')
print(introduction)
intro_seq = rvt.sequence(order='fixed', components = [introduction]) 
print(intro_seq) 

{
    "path": "vis-decode-retrieve-value/assets/introduction.md",
    "response": [],
    "type": "markdown"
}
{
    "components": [
        "introduction"
    ],
    "order": "fixed"
}


In [ ]:
# # rvt.sequence(order = 'random').from_data([(1, 2), (2, 3)])
# # asdict([1, 23])
# new_df = pd.DataFrame({"id": [i + 1 for i in range(10)]}, {"val": [i + 1 for i in range(10)]})
# print(new_df)
# new_df.to_csv('data.csv', index=False)
# print(rvt.data("data.csv"))

In [31]:
response = rvt.response(
    id = "retrieve_value", 
    prompt = 'Your selected answer', 
    location = 'belowStimulus', 
    type = 'numerical', 
    required = True,
)

In [43]:
n_rows = 25

new_df = pd.DataFrame({"id": [i + 1 for i in range(n_rows)]})
# print(new_df)
new_df.to_csv('data.csv', index=False)
# print(rvt.data("data.csv"))

data_sequence = rvt.sequence(order = 'random', numSamples=n_rows).from_data(rvt.data("data.csv"))
print(data_sequence)

{
    "components": [
        "place-holder-component_id:1",
        "place-holder-component_id:2",
        "place-holder-component_id:3",
        "place-holder-component_id:4",
        "place-holder-component_id:5",
        "place-holder-component_id:6",
        "place-holder-component_id:7",
        "place-holder-component_id:8",
        "place-holder-component_id:9",
        "place-holder-component_id:10",
        "place-holder-component_id:11",
        "place-holder-component_id:12",
        "place-holder-component_id:13",
        "place-holder-component_id:14",
        "place-holder-component_id:15",
        "place-holder-component_id:16",
        "place-holder-component_id:17",
        "place-holder-component_id:18",
        "place-holder-component_id:19",
        "place-holder-component_id:20",
        "place-holder-component_id:21",
        "place-holder-component_id:22",
        "place-holder-component_id:23",
        "place-holder-component_id:24",
        "place-holder-compo

In [33]:
def retrieve_value_component_function(id): 
    """
    This function is used to generate the component for the retrieve value study. 
    """
    row = ret_df.filter(pl.col('id') == id)
    chart_spec_value = row.select(pl.col('chart_spec')).item()
    question = row.select(pl.col('question')).item()
    # print(json.loads(chart_spec_value))
    # print(question)
    # print(chart_spec_value)

    # get the spec for the given id 
    comp = rvt.component(
        component_name__ = f'retrieve_value_{id}',
        type = 'vega',
        response = [response],
        config = json.loads(chart_spec_value),
        instruction = f'{question}',
        instructionLocation = 'belowStimulus',
        withSidebar = False,
    )
    return comp

In [34]:
data_sequence.component(retrieve_value_component_function)

_WrappedComponentBlock(root=ComponentBlock(components=[ComponentBlock(components=['retrieve_value_25', 'retrieve_value_24', 'retrieve_value_23', 'retrieve_value_22', 'retrieve_value_21', 'retrieve_value_20', 'retrieve_value_19', 'retrieve_value_18', 'retrieve_value_17', 'retrieve_value_16', 'retrieve_value_15', 'retrieve_value_14', 'retrieve_value_13', 'retrieve_value_12', 'retrieve_value_11', 'retrieve_value_10', 'retrieve_value_9', 'retrieve_value_8', 'retrieve_value_7', 'retrieve_value_6', 'retrieve_value_5', 'retrieve_value_4', 'retrieve_value_3', 'retrieve_value_2', 'retrieve_value_1'], id=None, interruptions=None, numSamples=25.0, order=<Order.random: 'random'>, skip=None), 'introduction'], id=None, interruptions=None, numSamples=None, order=<Order.fixed: 'fixed'>, skip=None), component_objects__=[_WrappedComponent(component_name__='retrieve_value_25', base__=None, context__=None, metadata__=None, root=VegaComponent(root=VegaComponentConfig(allowFailedTraining=None, config={'conf

In [35]:
print(data_sequence)
# print(data_sequence.get_components()[0])

{
    "components": [
        {
            "components": [
                "retrieve_value_25",
                "retrieve_value_24",
                "retrieve_value_23",
                "retrieve_value_22",
                "retrieve_value_21",
                "retrieve_value_20",
                "retrieve_value_19",
                "retrieve_value_18",
                "retrieve_value_17",
                "retrieve_value_16",
                "retrieve_value_15",
                "retrieve_value_14",
                "retrieve_value_13",
                "retrieve_value_12",
                "retrieve_value_11",
                "retrieve_value_10",
                "retrieve_value_9",
                "retrieve_value_8",
                "retrieve_value_7",
                "retrieve_value_6",
                "retrieve_value_5",
                "retrieve_value_4",
                "retrieve_value_3",
                "retrieve_value_2",
                "retrieve_value_1"
            ],
          

In [44]:
main_sequence = rvt.sequence(order='fixed',components=[introduction]) + data_sequence

study = rvt.studyConfig(
    schema="https://raw.githubusercontent.com/revisit-studies/study/v2.3.1/src/parser/StudyConfigSchema.json",
    uiConfig=ui_config,
    studyMetadata=study_metadata,
    sequence=main_sequence,
    importedLibraries = ['virtual-chinrest']
)
print(study)

{
    "$schema": "https://raw.githubusercontent.com/revisit-studies/study/v2.3.1/src/parser/StudyConfigSchema.json",
    "components": {
        "introduction": {
            "path": "vis-decode-retrieve-value/assets/introduction.md",
            "response": [],
            "type": "markdown"
        },
        "place-holder-component_id:1": {
            "meta": {
                "id": 1
            },
            "response": [],
            "type": "questionnaire"
        },
        "place-holder-component_id:2": {
            "meta": {
                "id": 2
            },
            "response": [],
            "type": "questionnaire"
        },
        "place-holder-component_id:3": {
            "meta": {
                "id": 3
            },
            "response": [],
            "type": "questionnaire"
        },
        "place-holder-component_id:4": {
            "meta": {
                "id": 4
            },
            "response": [],
            "type": "questionnaire

In [37]:
# code if we are to use the rvt_server 

# process = rvt_server.serve()
# process.terminate()
# w = rvt.widget(study, server = True)

In [38]:
str(study)
# print(study)

'{\n    "$schema": "https://raw.githubusercontent.com/revisit-studies/study/v2.3.1/src/parser/StudyConfigSchema.json",\n    "components": {\n        "retrieve_value_25": {\n            "config": {\n                "config": {\n                    "view": {\n                        "continuousWidth": 300,\n                        "continuousHeight": 300\n                    },\n                    "axis": {\n                        "grid": false\n                    },\n                    "bar": {\n                        "color": "gray"\n                    }\n                },\n                "data": {\n                    "name": "data-09e8949dda9fa38ba243975ef17b5f71"\n                },\n                "mark": {\n                    "type": "point",\n                    "color": "gray",\n                    "filled": true,\n                    "size": 300\n                },\n                "encoding": {\n                    "x": {\n                        "axis": {\n         

## Save study 

In [39]:
# write out the study configuration 

# Write directly to file
with open('public/vis-decode-retrieve-value/config.json', 'w', encoding='utf-8') as f:
    # json.write(str(study), f, indent=2, ensure_ascii=False)
    f.write(str(study))